# Data Manipulation

The notebook shows how to join two tables on Trino to create a new table. Such a table can then be used for further analysis or for creating visualizations in Apache Superset.

In [1]:
import os
import pathlib
from dotenv import load_dotenv
import trino
import pandas as pd

### Injecting Credentials

In order to run this notebook, we need credentials to connect with the Trino server to retrieve and create tables.

In an automated environment, the credentials can be specified in a pipeline's environment variables or through Openshift secrets. 

For running the notebook in a local environment, we will define them as environment variables in a `credentials.env` file, and load them using dotenv. An example of what the contents of `credentials.env` could look like is shown below

```
# trino credentials
TRINO_USER=xxx
TRINO_PASSWD=xxx
TRINO_HOST=trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org
TRINO_PORT=443
```

In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

# Join two tables on Trino

In [3]:
# Create an Trino client
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(
        os.environ["TRINO_USER"], os.environ["TRINO_PASSWD"]
    ),
    host=os.environ["TRINO_HOST"],
    port=int(os.environ["TRINO_PORT"]),
    http_scheme="https",
    verify=True,
)
cur = conn.cursor()

In [4]:
# Generate column names for df_emissions table
# and remove isin column to avoid duplication
# of key column in the join operation
cur.execute("show columns from default.urgentem.itr_emissions_1")
res = cur.fetchall()
df_columns = [i[0] for i in res]
df_columns.remove("isin")
df_columns = ["default.urgentem.itr_emissions_1." + i for i in df_columns]
df_columns = ", ".join(df_columns)

In [5]:
# Write the join_query
join_query = f"CREATE TABLE if not exists default.urgentem.itr_emissions_joined AS\
              SELECT default.urgentem.itr_emissions_2.*, {df_columns} \
              FROM default.urgentem.itr_emissions_2 \
              LEFT JOIN default.urgentem.itr_emissions_1 \
              ON default.urgentem.itr_emissions_1.isin=default.urgentem.itr_emissions_2.isin"
join_query

'CREATE TABLE if not exists default.urgentem.itr_emissions_joined AS              SELECT default.urgentem.itr_emissions_2.*, default.urgentem.itr_emissions_1.company_name, default.urgentem.itr_emissions_1.target_type, default.urgentem.itr_emissions_1.scope, default.urgentem.itr_emissions_1.coverage_s1, default.urgentem.itr_emissions_1.coverage_s2, default.urgentem.itr_emissions_1.coverage_s3, default.urgentem.itr_emissions_1.reduction_ambition, default.urgentem.itr_emissions_1.base_year, default.urgentem.itr_emissions_1.end_year, default.urgentem.itr_emissions_1.start_year, default.urgentem.itr_emissions_1.base_year_ghg_emissions_s1_tco2e, default.urgentem.itr_emissions_1.base_year_ghg_emissions_s1s2_tco2e, default.urgentem.itr_emissions_1.base_year_ghg_emissions_s3_tco2e, default.urgentem.itr_emissions_1.achieved_reduction               FROM default.urgentem.itr_emissions_2               LEFT JOIN default.urgentem.itr_emissions_1               ON default.urgentem.itr_emissions_1.isin=

In [6]:
# Execute the join query
cur.execute(join_query)
cur.fetchall()

[[0]]

In [7]:
# Check if the joined table is there
cur.execute("select * from default.urgentem.itr_emissions_joined LIMIT 5")
pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,BANCO SANTANDER,ES0113900J37,5705946,SAN SM,1,2,Spain,Europe,693.1,4.1,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
1,BARCLAYS PLC,GB0031348658,3134865,BARC LN,1,3,United Kingdom,Europe,709.2,9.7,...,1.0,None,0.37,2018.0,2025.0,2018.0,None,"282,593","1,412,965",0.0
2,BAYER AG-REG,DE000BAY0017,5069211,BAYN GR,1,8,Germany,Europe,385.5,69.2,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
3,ENEL SPA,IT0003128367,7144569,ENEL IM,1,4,Italy,Europe,1557.4,1116.2,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
4,GAZPROM,RU0007661625,B59L4L7,GAZP RM,2,0,Russian Federation,Europe,6033.4,2003.8,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN


# Conclusion

In this notebook, we showed how to read from existing Trino tables and perform a basic join operation on them to create a new table. The table can now be used in a Superset dashboard for visualization. 